# CLASIFICACION IMAGENES

## CAPITULO I. Cargue y transformación de las imagenes

In [51]:
import torchvision.transforms as transforms
import numpy as np

from sklearn.metrics import confusion_matrix
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

import torch

In [61]:
pipeline = transforms.Compose([
    transforms.Grayscale(num_output_channels= 1),   #las imagenes se pasan a escala de grises a 1 solo canal es decir blanco y negro
    transforms.Resize((400, 200)),                  #se usa un tamaño un poco mas grande de lo habitual para tratar de facilitar la deteccion de las palabras
    transforms.RandomHorizontalFlip(),              #se realiza un traslado horizontal
    transforms.RandomRotation(45),                   #rotacion de 45 grados
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))             #normaliza la serie de datos con media 0 y varianza 1
])

pipelineValidacion = transforms.Compose([
    transforms.Grayscale(num_output_channels= 1),   
    transforms.Resize((400, 200)),                    
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

baseImagenes = ImageFolder(root="./Imagenes", transform=pipeline)
baseImagenesValidacion = ImageFolder(root="./Imagenes_validacion", transform=pipelineValidacion)

In [62]:
tamanoEntrenamiento = int(0.8 * len(baseImagenes))
tamanoTesteo = len(baseImagenes) - tamanoEntrenamiento

baseEntrenamiento, baseTesteo = random_split(baseImagenes, [tamanoEntrenamiento, tamanoTesteo])

cargueEntrenammiento = DataLoader(baseEntrenamiento, batch_size=30, shuffle=True)
cargueTesteo = DataLoader(baseTesteo, batch_size=30, shuffle=False)
cargueValidacion = DataLoader(baseImagenesValidacion, batch_size=30, shuffle=False)


In [5]:
class redNeuronal(torch.nn.Module):
    def __init__(self, clases):
        super().__init__()
        self.feature_extractor = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, padding=1), #Entran imagenes a 1 canal se aplican 32 filtros de tamaño 3x3 
            torch.nn.ELU(),                                   #funcion de activacion exponencial
            torch.nn.MaxPool2d(kernel_size=2),                #reduce la dimensionalidad en 2 para mayor eficiencia
            torch.nn.Conv2d(32, 128, kernel_size=3, padding=1), 
            torch.nn.ELU(),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Flatten(),                               #salida de la red 
        )
        self.classifier = torch.nn.Linear(128 * 100 * 50, clases)
    
    def forward(self, x):  
        x = self.feature_extractor(x)                   #pasa el input por las capas
        x = self.classifier(x)                          #realiza la clasificacion
        return x

In [6]:
modelo = redNeuronal(clases=2)
criterio = torch.nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(modelo.parameters(), lr=0.001)

epochs = 10
dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo.to(dispositivo)

for epoch in range(epochs):
    modelo.train()
    perdidaEpoch = 0
    for imagenes, etiquetas in cargueEntrenammiento:
        imagenes, etiquetas = imagenes.to(dispositivo), etiquetas.to(dispositivo)
        optimizador.zero_grad()
        resultados = modelo(imagenes)
        perdida = criterio(resultados, etiquetas)
        perdida.backward()
        optimizador.step()
        
        perdidaEpoch += perdida.item()
    print(f"Epoch {epoch+1}/{epochs}, perdida: {perdidaEpoch/len(cargueEntrenammiento):.4f}")

print("Entrenamiento completado")


Epoch 1/10, perdida: 34.1367
Epoch 2/10, perdida: 9.4573
Epoch 3/10, perdida: 5.0476
Epoch 4/10, perdida: 3.4368
Epoch 5/10, perdida: 2.8265
Epoch 6/10, perdida: 0.8697
Epoch 7/10, perdida: 0.4995
Epoch 8/10, perdida: 2.1170
Epoch 9/10, perdida: 1.5993
Epoch 10/10, perdida: 1.7419
Entrenamiento completado


In [55]:
def validarImagenes(modelo, setImagenes):
    modelo.eval() 
    pred = []
    real = []
    with torch.no_grad():
        for imagenes, etiquetas in setImagenes:
            imagenes, etiquetas = imagenes.to(dispositivo), etiquetas.to(dispositivo)
            resultados = modelo(imagenes)
            _, prediccion = torch.max(resultados, 1)
            pred.extend(prediccion.tolist())
            real.extend(etiquetas.tolist())
    
    pred = np.array(pred).reshape(-1,)
    real = np.array(real).reshape(-1,)
    return confusion_matrix(real, pred)


In [58]:
#Testeo
matrizConfusionTesteo = validarImagenes(modelo, cargueTesteo)
print(matrizConfusionTesteo)

[[30  5]
 [ 0 15]]


In [64]:
#validacion
matrizConfusionValidacion = validarImagenes(modelo, cargueValidacion)
print(matrizConfusionValidacion)

[[1 0]
 [0 2]]


50